### **Metode IQR untuk Menghilangkan Outlier pada Dataset Indonesia Reading Interest**

Metode **Interquartile Range (IQR)** adalah teknik yang efektif untuk membersihkan data dari nilai-nilai ekstrem (pencilan atau *outlier*). Kode ini akan menunjukkan bagaimana cara kerjanya pada beberapa kolom numerik di dataset Indonesia Reading Interest.

In [ ]:
# Mengimpor library yang dibutuhkan
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Mengabaikan peringatan untuk tampilan yang lebih bersih
warnings.filterwarnings('ignore')

# Mengatur style plot
plt.style.use('seaborn-darkgrid')

In [ ]:
# Memuat dataset
# Pastikan Anda sudah mengunggah file TGM 2020-2023_eng.csv ke Google Colab
file_path = 'TGM 2020-2023_eng.csv'
df = pd.read_csv(file_path, delimiter=';', encoding='UTF-8-SIG')

# --- Data Preprocessing: Konversi dan Penghapusan Missing Value ---
# Mengubah kolom yang relevan menjadi tipe numerik. 'errors=coerce' akan mengubah nilai yang tidak bisa dikonversi menjadi NaN
df['Reading Frequency per week'] = pd.to_numeric(df['Reading Frequency per week'], errors='coerce')
df['Number of Readings per Quarter'] = pd.to_numeric(df['Number of Readings per Quarter'], errors='coerce')
df['Daily Reading Duration (in minutes)'] = pd.to_numeric(df['Daily Reading Duration (in minutes)'], errors='coerce')
df['Internet Access Frequency per Week'] = pd.to_numeric(df['Internet Access Frequency per Week'], errors='coerce')
df['Daily Internet Duration (in minutes)'] = pd.to_numeric(df['Daily Internet Duration (in minutes)'], errors='coerce')

# Menghapus baris dengan missing values (NaN) yang dihasilkan dari konversi
df.dropna(inplace=True)

print("Informasi Dataset setelah preprocessing:\n")
print(df.info())
print("\n5 baris pertama dataset:\n")
print(df.head())

### **Pembersihan Outlier pada 'Daily Reading Duration (in minutes)'**
Kita akan menggunakan metode IQR untuk membersihkan kolom durasi membaca harian. Nilai yang sangat tinggi atau sangat rendah akan dihapus.

In [ ]:
# Visualisasi histogram sebelum pembersihan outlier
plt.figure(figsize=(12, 6))
sns.histplot(df['Daily Reading Duration (in minutes)'], bins=50, kde=True, color='blue')
plt.title('Distribusi Sebelum Pembersihan Outlier: Durasi Membaca Harian')
plt.xlabel('Durasi (menit)')
plt.ylabel('Frekuensi')
plt.show()

# Menghitung Kuartil (Q1 dan Q3) dan IQR
Q1 = df['Daily Reading Duration (in minutes)'].quantile(0.25)
Q3 = df['Daily Reading Duration (in minutes)'].quantile(0.75)
IQR = Q3 - Q1

# Menghitung batas bawah dan batas atas untuk mendeteksi outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1: {Q1}")
print(f"Q3: {Q3}")
print(f"IQR: {IQR}")
print(f"Batas Bawah: {lower_bound}")
print(f"Batas Atas: {upper_bound}")

# Memfilter DataFrame untuk menghapus outlier
df_filtered_reading = df[(df['Daily Reading Duration (in minutes)'] >= lower_bound) & 
                         (df['Daily Reading Duration (in minutes)'] <= upper_bound)]

print(f"\nJumlah data sebelum filter: {len(df)}")
print(f"Jumlah data setelah filter: {len(df_filtered_reading)}")

# Visualisasi histogram setelah pembersihan outlier
plt.figure(figsize=(12, 6))
sns.histplot(df_filtered_reading['Daily Reading Duration (in minutes)'], bins=50, kde=True, color='red')
plt.title('Distribusi Setelah Pembersihan Outlier: Durasi Membaca Harian')
plt.xlabel('Durasi (menit)')
plt.ylabel('Frekuensi')
plt.show()

### **Pembersihan Outlier pada 'Daily Internet Duration (in minutes)'**
Kita akan mengulangi proses yang sama untuk kolom durasi internet harian.

In [ ]:
# Visualisasi histogram sebelum pembersihan outlier
plt.figure(figsize=(12, 6))
sns.histplot(df['Daily Internet Duration (in minutes)'], bins=50, kde=True, color='green')
plt.title('Distribusi Sebelum Pembersihan Outlier: Durasi Internet Harian')
plt.xlabel('Durasi (menit)')
plt.ylabel('Frekuensi')
plt.show()

# Menghitung Kuartil (Q1 dan Q3) dan IQR
Q1_internet = df['Daily Internet Duration (in minutes)'].quantile(0.25)
Q3_internet = df['Daily Internet Duration (in minutes)'].quantile(0.75)
IQR_internet = Q3_internet - Q1_internet

# Menghitung batas bawah dan batas atas
lower_bound_internet = Q1_internet - 1.5 * IQR_internet
upper_bound_internet = Q3_internet + 1.5 * IQR_internet

print(f"Q1: {Q1_internet}")
print(f"Q3: {Q3_internet}")
print(f"IQR: {IQR_internet}")
print(f"Batas Bawah: {lower_bound_internet}")
print(f"Batas Atas: {upper_bound_internet}")

# Memfilter DataFrame untuk menghapus outlier
df_filtered_internet = df[(df['Daily Internet Duration (in minutes)'] >= lower_bound_internet) & 
                          (df['Daily Internet Duration (in minutes)'] <= upper_bound_internet)]

print(f"\nJumlah data sebelum filter: {len(df)}")
print(f"Jumlah data setelah filter: {len(df_filtered_internet)}")

# Visualisasi histogram setelah pembersihan outlier
plt.figure(figsize=(12, 6))
sns.histplot(df_filtered_internet['Daily Internet Duration (in minutes)'], bins=50, kde=True, color='purple')
plt.title('Distribusi Setelah Pembersihan Outlier: Durasi Internet Harian')
plt.xlabel('Durasi (menit)')
plt.ylabel('Frekuensi')
plt.show()